In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [2]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Umbria
df_er = df_comuni[df_comuni['cod_regione'] == '10']

In [3]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/tmp/ipykernel_1751/3372484651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/tmp/ipykernel_1751/3372484651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
393,103040580420,POGGIODOMO,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0420,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
417,103040800060,ATTIGLIANO,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0060,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
356,103040580050,CAMPELLO SUL CLITUNNO,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0050,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...


In [4]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [5]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
   print(f"\nProcessing record {idx + 1}")
   print(f"URL: {url}")

   response = requests.get(url, headers=headers)
   print(f"Response status code: {response.status_code}")
   
   data = response.json()

   comune = df_er.iloc[idx]['desc']
   provincia = df_er.iloc[idx]['provincia']
   regione = df_er.iloc[idx]['regione']
   url = df_er.iloc[idx]['url']

   print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

   # Start with basic comune info
   comune_info = {
       'comune': comune,
       'provincia': provincia,
       'regione': regione,
       'url': url
   }
   
   # Add candidate totals
   if 'cand' in data:
       for cand in data['cand']:
           nome_completo = cand['nome'] + " " + cand['cogn']
           comune_info[f"{nome_completo}_total"] = cand['voti']
           comune_info[f"{nome_completo}_perc"] = cand['perc']
           print(f"{nome_completo}: {cand['voti']} votes ({cand['perc']}%)")

   comuni_data.append(comune_info)
   
   print(f"Processed {comune}")
   sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0010


Response status code: 200
Processing data for Comune: ASSISI, Provincia: TERNI
STEFANIA PROIETTI: 6096 votes (49,27%)
DONATELLA TESEI: 5986 votes (48,38%)
MARCO RIZZO: 123 votes (0,99%)
MARTINA LEONARDI: 52 votes (0,42%)
GIUSEPPE PAOLONE: 32 votes (0,26%)
ELIA FRANCESCO FIORINI: 25 votes (0,20%)
MORENO PASQUINELLI: 23 votes (0,19%)
FABRIZIO PIGNALBERI: 21 votes (0,17%)
GIUSEPPE TRITTO: 15 votes (0,12%)
Processed ASSISI

Processing record 2
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0020


Response status code: 200
Processing data for Comune: BASTIA UMBRA, Provincia: TERNI
STEFANIA PROIETTI: 4378 votes (51,18%)
DONATELLA TESEI: 4005 votes (46,82%)
MARCO RIZZO: 74 votes (0,87%)
MARTINA LEONARDI: 27 votes (0,32%)
GIUSEPPE PAOLONE: 21 votes (0,25%)
GIUSEPPE TRITTO: 20 votes (0,23%)
MORENO PASQUINELLI: 14 votes (0,16%)
ELIA FRANCESCO FIORINI: 8 votes (0,09%)
FABRIZIO PIGNALBERI: 7 votes (0,08%)
Processed BASTIA UMBRA

Processing record 3
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0030


Response status code: 200
Processing data for Comune: BETTONA, Provincia: TERNI
DONATELLA TESEI: 981 votes (53,72%)
STEFANIA PROIETTI: 803 votes (43,98%)
MARCO RIZZO: 26 votes (1,42%)
ELIA FRANCESCO FIORINI: 5 votes (0,27%)
GIUSEPPE PAOLONE: 3 votes (0,16%)
MARTINA LEONARDI: 3 votes (0,16%)
GIUSEPPE TRITTO: 2 votes (0,11%)
MORENO PASQUINELLI: 2 votes (0,11%)
FABRIZIO PIGNALBERI: 1 votes (0,05%)
Processed BETTONA

Processing record 4
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0040


Response status code: 200
Processing data for Comune: BEVAGNA, Provincia: TERNI
STEFANIA PROIETTI: 976 votes (49,92%)
DONATELLA TESEI: 935 votes (47,83%)
MARCO RIZZO: 16 votes (0,82%)
MARTINA LEONARDI: 12 votes (0,61%)
GIUSEPPE TRITTO: 9 votes (0,46%)
MORENO PASQUINELLI: 4 votes (0,20%)
ELIA FRANCESCO FIORINI: 2 votes (0,10%)
FABRIZIO PIGNALBERI: 1 votes (0,05%)
GIUSEPPE PAOLONE: 0 votes (0%)
Processed BEVAGNA

Processing record 5
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0050


...

Response status code: 200
Processing data for Comune: PORANO, Provincia: TERNI
STEFANIA PROIETTI: 439 votes (52,70%)
DONATELLA TESEI: 368 votes (44,18%)
MARTINA LEONARDI: 8 votes (0,96%)
MORENO PASQUINELLI: 8 votes (0,96%)
MARCO RIZZO: 5 votes (0,60%)
ELIA FRANCESCO FIORINI: 4 votes (0,48%)
GIUSEPPE PAOLONE: 1 votes (0,12%)
GIUSEPPE TRITTO: 0 votes (0%)
FABRIZIO PIGNALBERI: 0 votes (0%)
Processed PORANO

Processing record 89
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0290


Response status code: 200
Processing data for Comune: SAN GEMINI, Provincia: TERNI
STEFANIA PROIETTI: 1036 votes (51,31%)
DONATELLA TESEI: 932 votes (46,16%)
MARCO RIZZO: 19 votes (0,94%)
MARTINA LEONARDI: 12 votes (0,59%)
MORENO PASQUINELLI: 6 votes (0,30%)
ELIA FRANCESCO FIORINI: 5 votes (0,25%)
GIUSEPPE PAOLONE: 3 votes (0,15%)
GIUSEPPE TRITTO: 3 votes (0,15%)
FABRIZIO PIGNALBERI: 3 votes (0,15%)
Processed SAN GEMINI

Processing record 90
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0300


Response status code: 200
Processing data for Comune: SAN VENANZO, Provincia: TERNI
STEFANIA PROIETTI: 595 votes (53,46%)
DONATELLA TESEI: 497 votes (44,65%)
MARCO RIZZO: 13 votes (1,17%)
MARTINA LEONARDI: 3 votes (0,27%)
ELIA FRANCESCO FIORINI: 3 votes (0,27%)
GIUSEPPE TRITTO: 1 votes (0,09%)
MORENO PASQUINELLI: 1 votes (0,09%)
GIUSEPPE PAOLONE: 0 votes (0%)
FABRIZIO PIGNALBERI: 0 votes (0%)
Processed SAN VENANZO

Processing record 91
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0310


Response status code: 200
Processing data for Comune: STRONCONE, Provincia: TERNI
DONATELLA TESEI: 1008 votes (56,88%)
STEFANIA PROIETTI: 705 votes (39,79%)
MARTINA LEONARDI: 21 votes (1,19%)
MARCO RIZZO: 19 votes (1,07%)
GIUSEPPE PAOLONE: 10 votes (0,56%)
GIUSEPPE TRITTO: 4 votes (0,23%)
MORENO PASQUINELLI: 3 votes (0,17%)
ELIA FRANCESCO FIORINI: 1 votes (0,06%)
FABRIZIO PIGNALBERI: 1 votes (0,06%)
Processed STRONCONE

Processing record 92
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0320


Response status code: 200
Processing data for Comune: TERNI, Provincia: TERNI
STEFANIA PROIETTI: 19356 votes (49,85%)
DONATELLA TESEI: 18298 votes (47,12%)
MARCO RIZZO: 459 votes (1,18%)
MARTINA LEONARDI: 374 votes (0,96%)
GIUSEPPE PAOLONE: 104 votes (0,27%)
MORENO PASQUINELLI: 80 votes (0,21%)
ELIA FRANCESCO FIORINI: 75 votes (0,19%)
GIUSEPPE TRITTO: 58 votes (0,15%)
FABRIZIO PIGNALBERI: 26 votes (0,07%)
Processed TERNI

Creating DataFrame...

Sample of processed data:
                  comune provincia regione  \
0                 ASSISI     TERNI  UMBRIA   
1           BASTIA UMBRA     TERNI  UMBRIA   
2                BETTONA     TERNI  UMBRIA   
3                BEVAGNA     TERNI  UMBRIA   
4  CAMPELLO SUL CLITUNNO     TERNI  UMBRIA   

                                                 url  STEFANIA PROIETTI_total  \
0  https://elezioni.interno.gov.it/risultati/2024...                     6096   
1  https://elezioni.interno.gov.it/risultati/2024...                     4378   
2  ht

In [6]:
df_data

,comune,provincia,regione,url,STEFANIA PROIETTI_total,STEFANIA PROIETTI_perc,DONATELLA TESEI_total,DONATELLA TESEI_perc,MARCO RIZZO_total,MARCO RIZZO_perc,...,GIUSEPPE PAOLONE_total,GIUSEPPE PAOLONE_perc,ELIA FRANCESCO FIORINI_total,ELIA FRANCESCO FIORINI_perc,MORENO PASQUINELLI_total,MORENO PASQUINELLI_perc,FABRIZIO PIGNALBERI_total,FABRIZIO PIGNALBERI_perc,GIUSEPPE TRITTO_total,GIUSEPPE TRITTO_perc
0,ASSISI,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,6096,"49,27",5986,"48,38",123,"0,99",...,32,"0,26",25,"0,20",23,"0,19",21,"0,17",15,"0,12"
1,BASTIA UMBRA,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,4378,"51,18",4005,"46,82",74,"0,87",...,21,"0,25",8,"0,09",14,"0,16",7,"0,08",20,"0,23"
2,BETTONA,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,803,"43,98",981,"53,72",26,"1,42",...,3,"0,16",5,"0,27",2,"0,11",1,"0,05",2,"0,11"
3,BEVAGNA,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,976,"49,92",935,"47,83",16,"0,82",...,0,0,2,"0,10",4,"0,20",1,"0,05",9,"0,46"
4,CAMPELLO SUL CLITUNNO,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,450,"50,00",420,"46,67",12,"1,33",...,0,0,3,"0,33",7,"0,78",0,0,4,"0,44"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,PORANO,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,439,"52,70",368,"44,18",5,"0,60",...,1,"0,12",4,"0,48",8,"0,96",0,0,0,0
88,SAN GEMINI,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,1036,"51,31",932,"46,16",19,"0,94",...,3,"0,15",5,"0,25",6,"0,30",3,"0,15",3,"0,15"
89,SAN VENANZO,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,595,"53,46",497,"44,65",13,"1,17",...,0,0,3,"0,27",1,"0,09",0,0,1,"0,09"
90,STRONCONE,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,705,"39,79",1008,"56,88",19,"1,07",...,10,"0,56",1,"0,06",3,"0,17",1,"0,06",4,"0,23"


In [7]:
df_data['perc_diff'] = round(((df_data['STEFANIA PROIETTI_total'] - df_data['DONATELLA TESEI_total']) / df_data['DONATELLA TESEI_total']) * 100, 2)

In [8]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,comune,provincia,regione,url,STEFANIA PROIETTI_total,STEFANIA PROIETTI_perc,DONATELLA TESEI_total,DONATELLA TESEI_perc,MARCO RIZZO_total,MARCO RIZZO_perc,...,GIUSEPPE PAOLONE_perc,ELIA FRANCESCO FIORINI_total,ELIA FRANCESCO FIORINI_perc,MORENO PASQUINELLI_total,MORENO PASQUINELLI_perc,FABRIZIO PIGNALBERI_total,FABRIZIO PIGNALBERI_perc,GIUSEPPE TRITTO_total,GIUSEPPE TRITTO_perc,perc_diff
0,Assisi,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,6096,"49,27",5986,"48,38",123,"0,99",...,"0,26",25,"0,20",23,"0,19",21,"0,17",15,"0,12",1.84
1,Bastia Umbra,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,4378,"51,18",4005,"46,82",74,"0,87",...,"0,25",8,"0,09",14,"0,16",7,"0,08",20,"0,23",9.31
2,Bettona,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,803,"43,98",981,"53,72",26,"1,42",...,"0,16",5,"0,27",2,"0,11",1,"0,05",2,"0,11",-18.14
3,Bevagna,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,976,"49,92",935,"47,83",16,"0,82",...,0,2,"0,10",4,"0,20",1,"0,05",9,"0,46",4.39
4,Campello Sul Clitunno,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,450,"50,00",420,"46,67",12,"1,33",...,0,3,"0,33",7,"0,78",0,0,4,"0,44",7.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Porano,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,439,"52,70",368,"44,18",5,"0,60",...,"0,12",4,"0,48",8,"0,96",0,0,0,0,19.29
88,San Gemini,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,1036,"51,31",932,"46,16",19,"0,94",...,"0,15",5,"0,25",6,"0,30",3,"0,15",3,"0,15",11.16
89,San Venanzo,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,595,"53,46",497,"44,65",13,"1,17",...,0,3,"0,27",1,"0,09",0,0,1,"0,09",19.72
90,Stroncone,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,705,"39,79",1008,"56,88",19,"1,07",...,"0,56",1,"0,06",3,"0,17",1,"0,06",4,"0,23",-30.06


In [9]:
# Use .str.replace() to handle string replacement more safely
df_data['comune'] = df_data['comune'].str.replace("Citta'", "Città", regex=False)


In [10]:
df_data.to_csv('../output/risultati_um.csv', index=False, encoding='UTF-8-sig')